<a href="https://colab.research.google.com/github/lamakye7/AdventureWorks-Sale/blob/main/OmdenaAI_Weather_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install meteostat


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import warnings


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from meteostat import Stations, Hourly




In [3]:
df = pd.read_parquet("ist_traffic_density_rev01.gz")

In [5]:
df.head()

,DATE_TIME,LATITUDE,LONGITUDE,GEOHASH,MINIMUM_SPEED,MAXIMUM_SPEED,AVERAGE_SPEED,NUMBER_OF_VEHICLES,YEAR,MONTH,DAY,HOUR,DAYOFWEEK
0,2020-01-01,41.080627,28.811646,sxk3xw,135,18,81,132,2020,1,1,0,2
1,2020-01-01,40.987244,29.108276,sxk9nm,143,10,73,162,2020,1,1,0,2
2,2020-01-01,41.003723,29.097290,sxk9q0,128,6,50,110,2020,1,1,0,2
3,2020-01-01,40.998230,28.679810,sxk3hx,111,22,68,101,2020,1,1,0,2
4,2020-01-01,41.042175,28.020630,sx7cmx,99,99,99,1,2020,1,1,0,2


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66146858 entries, 0 to 66146857
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   DATE_TIME           datetime64[ns]
 1   LATITUDE            float32       
 2   LONGITUDE           float32       
 3   GEOHASH             object        
 4   MINIMUM_SPEED       uint8         
 5   MAXIMUM_SPEED       uint8         
 6   AVERAGE_SPEED       uint8         
 7   NUMBER_OF_VEHICLES  uint16        
 8   YEAR                uint16        
 9   MONTH               uint8         
 10  DAY                 uint8         
 11  HOUR                uint8         
 12  DAYOFWEEK           uint8         
dtypes: datetime64[ns](1), float32(2), object(1), uint16(2), uint8(7)
memory usage: 2.2+ GB


In [5]:
df.describe()

,LATITUDE,LONGITUDE,MINIMUM_SPEED,MAXIMUM_SPEED,AVERAGE_SPEED,NUMBER_OF_VEHICLES,YEAR,MONTH,DAY,HOUR,DAYOFWEEK
count,6.614686e+07,6.614686e+07,6.614686e+07,6.614686e+07,6.614686e+07,6.614686e+07,6.614686e+07,6.614686e+07,6.614686e+07,6.614686e+07,6.614686e+07
mean,4.106355e+01,2.888414e+01,9.776182e+01,2.535347e+01,5.642454e+01,7.287544e+01,2.021205e+03,6.149557e+00,1.571489e+01,1.164007e+01,2.998898e+00
std,1.002443e-01,3.546127e-01,3.491571e+01,2.444912e+01,2.468059e+01,9.505358e+01,9.793281e-01,3.522747e+00,8.792143e+00,6.882054e+00,1.997217e+00
min,4.074554e+01,2.796570e+01,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,2.020000e+03,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,4.099823e+01,2.869080e+01,7.000000e+01,6.000000e+00,3.400000e+01,1.300000e+01,2.020000e+03,3.000000e+00,8.000000e+00,6.000000e+00,1.000000e+00
50%,4.105865e+01,2.893250e+01,9.700000e+01,1.300000e+01,5.600000e+01,3.900000e+01,2.021000e+03,6.000000e+00,1.600000e+01,1.200000e+01,3.000000e+00
75%,4.113007e+01,2.914124e+01,1.240000e+02,4.600000e+01,7.900000e+01,9.300000e+01,2.022000e+03,9.000000e+00,2.300000e+01,1.800000e+01,5.000000e+00
max,4.164642e+01,2.988831e+01,2.550000e+02,2.430000e+02,2.430000e+02,1.351000e+03,2.023000e+03,1.200000e+01,3.100000e+01,2.300000e+01,6.000000e+00


In [4]:
grouped_data = df.groupby(['LATITUDE', 'LONGITUDE'])["NUMBER_OF_VEHICLES"].sum().reset_index()
grouped_data

,LATITUDE,LONGITUDE,NUMBER_OF_VEHICLES
0,40.745544,29.415894,5
1,40.745544,29.426880,3
2,40.745544,29.437866,3
3,40.751038,27.965698,1
4,40.751038,29.415894,4
...,...,...,...
5009,41.635437,28.097534,1
5010,41.640930,28.020630,1
5011,41.640930,28.064575,2
5012,41.646423,28.064575,1


In [5]:
import folium
from folium.plugins import HeatMap
# Calculate the mean latitude and longitude
mean_lat = grouped_data['LATITUDE'].mean()
mean_lon = grouped_data['LONGITUDE'].mean()

# Create the base map
m = folium.Map(location=[mean_lat, mean_lon], zoom_start=10)

locations = grouped_data[['LATITUDE', 'LONGITUDE', 'NUMBER_OF_VEHICLES']].values.tolist()

# heatmap using the location
heat_map = HeatMap(locations, min_opacity=0.5, max_val=max(grouped_data['NUMBER_OF_VEHICLES']))
# add the heatmap to the base map
heat_map.add_to(m)


In [10]:
m

In [11]:
m.save("traffic_heatmap.html")


In [9]:

from datetime import datetime

start_date = datetime(2020,1,1)
end_date = datetime(2023,4,30)


latitude = mean_lat # mean latitude
longitude = mean_lon # mean longitude
# Find the nearest weather station based on the coordinates
stations = Stations()
station = stations.nearby(latitude, longitude).fetch(1)

# Check if a weather station was found for the given coordinates
if not station.empty:
  station_id = station.index[0]


# Use the Hourly class to fetch the hourly weather data for the specified date and station
  data = Hourly(station_id, start=start_date, end=end_date)
  hourly_data = data.fetch()

else:
  print("No weather station found for the coordinates")




In [10]:
hourly_data

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
time,,,,,,,,,,,
2020-01-01 00:00:00,5.5,1.8,77.0,NaN,NaN,300.0,5.4,NaN,1021.9,NaN,3.0
2020-01-01 01:00:00,6.0,2.1,76.0,NaN,NaN,310.0,9.4,NaN,1022.0,NaN,2.0
2020-01-01 02:00:00,5.0,-1.0,65.0,NaN,NaN,310.0,16.6,NaN,1021.0,NaN,2.0
2020-01-01 03:00:00,4.5,-2.6,60.0,NaN,NaN,320.0,11.2,NaN,1021.9,NaN,3.0
2020-01-01 04:00:00,5.0,-1.0,65.0,NaN,NaN,340.0,9.4,22.2,1021.0,NaN,3.0
...,...,...,...,...,...,...,...,...,...,...,...
2023-04-29 20:00:00,11.0,4.0,62.0,0.0,NaN,210.0,6.0,NaN,1015.0,NaN,1.0
2023-04-29 21:00:00,10.5,3.8,63.0,0.0,NaN,210.0,5.0,NaN,1015.3,NaN,1.0
2023-04-29 22:00:00,10.0,3.9,66.0,0.0,NaN,250.0,4.0,NaN,1014.0,NaN,1.0


In [12]:
hourly_data = hourly_data.reset_index()
hourly_data


,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
0,2020-01-01 00:00:00,5.5,1.8,77.0,NaN,NaN,300.0,5.4,NaN,1021.9,NaN,3.0
1,2020-01-01 01:00:00,6.0,2.1,76.0,NaN,NaN,310.0,9.4,NaN,1022.0,NaN,2.0
2,2020-01-01 02:00:00,5.0,-1.0,65.0,NaN,NaN,310.0,16.6,NaN,1021.0,NaN,2.0
3,2020-01-01 03:00:00,4.5,-2.6,60.0,NaN,NaN,320.0,11.2,NaN,1021.9,NaN,3.0
4,2020-01-01 04:00:00,5.0,-1.0,65.0,NaN,NaN,340.0,9.4,22.2,1021.0,NaN,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
29156,2023-04-29 20:00:00,11.0,4.0,62.0,0.0,NaN,210.0,6.0,NaN,1015.0,NaN,1.0
29157,2023-04-29 21:00:00,10.5,3.8,63.0,0.0,NaN,210.0,5.0,NaN,1015.3,NaN,1.0
29158,2023-04-29 22:00:00,10.0,3.9,66.0,0.0,NaN,250.0,4.0,NaN,1014.0,NaN,1.0
29159,2023-04-29 23:00:00,10.0,5.0,71.0,0.0,NaN,240.0,7.0,NaN,1014.0,NaN,1.0


In [13]:
hourly_data.isnull().sum()

time        0
temp        0
dwpt        0
rhum        0
prcp    21049
snow    29068
wdir        0
wspd        0
wpgt    28710
pres        0
tsun    29161
coco      240
dtype: int64

In [14]:
hourly_data =  hourly_data.drop(["prcp", "snow", "wpgt", "tsun"], axis= 1)
hourly_data.head

<bound method NDFrame.head of                      time  temp  dwpt  rhum   wdir  wspd    pres  coco
0     2020-01-01 00:00:00   5.5   1.8  77.0  300.0   5.4  1021.9   3.0
1     2020-01-01 01:00:00   6.0   2.1  76.0  310.0   9.4  1022.0   2.0
2     2020-01-01 02:00:00   5.0  -1.0  65.0  310.0  16.6  1021.0   2.0
3     2020-01-01 03:00:00   4.5  -2.6  60.0  320.0  11.2  1021.9   3.0
4     2020-01-01 04:00:00   5.0  -1.0  65.0  340.0   9.4  1021.0   3.0
...                   ...   ...   ...   ...    ...   ...     ...   ...
29156 2023-04-29 20:00:00  11.0   4.0  62.0  210.0   6.0  1015.0   1.0
29157 2023-04-29 21:00:00  10.5   3.8  63.0  210.0   5.0  1015.3   1.0
29158 2023-04-29 22:00:00  10.0   3.9  66.0  250.0   4.0  1014.0   1.0
29159 2023-04-29 23:00:00  10.0   5.0  71.0  240.0   7.0  1014.0   1.0
29160 2023-04-30 00:00:00  10.3   5.3  71.0  240.0   8.0  1014.2   1.0

[29161 rows x 8 columns]>

In [15]:
hourly_data

,time,temp,dwpt,rhum,wdir,wspd,pres,coco
0,2020-01-01 00:00:00,5.5,1.8,77.0,300.0,5.4,1021.9,3.0
1,2020-01-01 01:00:00,6.0,2.1,76.0,310.0,9.4,1022.0,2.0
2,2020-01-01 02:00:00,5.0,-1.0,65.0,310.0,16.6,1021.0,2.0
3,2020-01-01 03:00:00,4.5,-2.6,60.0,320.0,11.2,1021.9,3.0
4,2020-01-01 04:00:00,5.0,-1.0,65.0,340.0,9.4,1021.0,3.0
...,...,...,...,...,...,...,...,...
29156,2023-04-29 20:00:00,11.0,4.0,62.0,210.0,6.0,1015.0,1.0
29157,2023-04-29 21:00:00,10.5,3.8,63.0,210.0,5.0,1015.3,1.0
29158,2023-04-29 22:00:00,10.0,3.9,66.0,250.0,4.0,1014.0,1.0
29159,2023-04-29 23:00:00,10.0,5.0,71.0,240.0,7.0,1014.0,1.0


In [16]:
hourly_data.to_csv('OmdenaAI_Predicting Traffic Density_Weather_Date.csv',index=False)